<a href="https://colab.research.google.com/github/AlexH20/NLP_Sentiment_Methods/blob/main/Merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import csv
from datetime import datetime, timedelta, time
from google.colab import drive 
import sys
import re
import math
import holidays

drive.mount("/content/gdrive")

#Functions to concatenate text

def concatenate_text(g):
    return ' '.join(g.Text)

#Function to count words of text using regex

def count_words(Text):
    word_count = len(re.findall(r'\w+', Text))
    return word_count

def check_weekend_returnmonday(day):
    trading_day = day
    #Check if news article published later than 4pm. If so, then return next day
    if trading_day.hour >= 16:
        trading_day += timedelta(1)
    #Check if news article published on weekend. If so return monday 
    while trading_day.weekday() in holidays.WEEKEND:
        trading_day += timedelta(1)
    return trading_day

output_file_path = "gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/"
output_file_name = "data_whole_woS.csv"

data_text = pd.read_csv("gdrive/My Drive/Thesis/processed data/processdata_woSWandPS.csv")
df1 = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_year1.csv")
df2 = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_year2.csv")
df3 = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_year3.csv")
df4 = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_year4.csv")
df5 = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_year5.csv")

frames = [df1, df2, df3, df4, df5]

df = pd.concat(frames)

#Get both dfs to same date format. Later used for merging. Turn saturdays and sundays into mondays

data_text["Date"] = pd.to_datetime(data_text["Date"])

data_text["Date"] = [check_weekend_returnmonday(x) for x in data_text["Date"]]
data_text["Date"] = [x.strftime("%Y-%m-%d") for x in data_text["Date"]]

df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = [check_weekend_returnmonday(x) for x in df["Date"]]
df["Date"] = [x.strftime("%Y-%m-%d") for x in df["Date"]]

#Merge rows with multiple news article issue of the same company on a given day
new_df = data_text.groupby(["Date", "Ticker"]).apply(concatenate_text).to_frame(name="c_Text")
df = df.merge(new_df, how= "left", left_on = ["Date", "Ticker"], right_on = ["Date", "Ticker"])

df["c_Text"] = df["c_Text"].fillna(" ")

df["word_count"] = [count_words(x) for x in df.c_Text]

df.drop_duplicates(inplace = True)

df = df.rename(columns={'c_Text': 'Text'})

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(
        ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM",
         "pref_alpha", "CAR", "Text", "word_count"])
    for index, row in df.iterrows():
        writer.writerow(row)






Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
